In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.stats import rankdata
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Directories
input_dir = os.path.join(os.getcwd(), "IMF_data_by_countries")
output_dir = os.path.join(os.getcwd(), "IMF_data_by_countries_after_processing")
os.makedirs(output_dir, exist_ok=True)

# Ignore sheets
ignore_sheets = {"missing_indicators", "IMF_Indicators"}

# Indicator groups
households = ["S14", "S15"]
financials = ["S12"]
nonfinancials = ["S11"]

def cdf_transform(series):
    """Applies a rank-based (CDF) transformation."""
    ranks = rankdata(series)
    return ranks / (len(ranks) + 1)

def process_country(file_name):
    if not file_name.endswith(".xlsx"):
        return
    
    country_name = file_name.replace(".xlsx", "")
    file_path = os.path.join(input_dir, file_name)
    print(f"Processing {country_name}...")
    
    try:
        xls = pd.ExcelFile(file_path)
        sheets = xls.sheet_names
    except Exception as e:
        print(f"Error reading {file_name}: {e}")
        return
    
    indicator_sheets = [sheet for sheet in sheets if sheet not in ignore_sheets]
    if not indicator_sheets:
        print(f"No valid indicator sheets found for {country_name}. Skipping...")
        return
    
    data = {}
    for sheet in indicator_sheets:
        try:
            df = pd.read_excel(file_path, sheet_name=sheet)
            if 'date' not in df.columns or 'value' not in df.columns:
                continue
            df = df[['date', 'value']].rename(columns={'value': sheet})
            df['date'] = pd.to_datetime(df['date'])
            df.sort_values('date', inplace=True)
            data[sheet] = df
        except Exception as e:
            print(f"Error in sheet '{sheet}' for {country_name}: {e}")
            continue
    
    if not data:
        print(f"No valid data for {country_name}. Skipping...")
        return
    
    # Merge data by date
    merged_data = None
    for sheet, df in data.items():
        if merged_data is None:
            merged_data = df
        else:
            merged_data = pd.merge(merged_data, df, on='date', how='outer')
    
    merged_data.set_index('date', inplace=True)
    numeric_cols = merged_data.select_dtypes(include=[np.number]).columns
    
    # Apply CDF transformation
    for col in numeric_cols:
        merged_data[col] = cdf_transform(merged_data[col])
    
    # Subsetting into categories
    household_data = merged_data[[col for col in numeric_cols if any(h in col for h in households)]]
    financial_data = merged_data[[col for col in numeric_cols if any(f in col for f in financials)]]
    nonfinancial_data = merged_data[[col for col in numeric_cols if any(n in col for n in nonfinancials)]]
    
    # Apply PCA per subset
    for subset_name, subset_data in zip([
        "Households", "Financials", "Non-Financials"],
        [household_data, financial_data, nonfinancial_data]):
        
        if subset_data.empty:
            continue
        
        subset_data.fillna(subset_data.mean(), inplace=True)
        
        scaler = StandardScaler()
        scaled_data = scaler.fit_transform(subset_data)
        
        pca = PCA()
        pca.fit(scaled_data)
        eigenvalues = pca.explained_variance_
        selected_components = [i+1 for i, val in enumerate(eigenvalues) if val > 1]
        
        if not selected_components:
            continue
        
        pca = PCA(n_components=len(selected_components))
        scores = pca.fit_transform(scaled_data)
        loadings = pca.components_
        
        output_path = os.path.join(output_dir, country_name, subset_name)
        os.makedirs(output_path, exist_ok=True)
        
        # Save PCA results
        scores_df = pd.DataFrame(scores, columns=[f"PC{i}" for i in range(1, len(selected_components)+1)], index=subset_data.index)
        scores_df.to_csv(os.path.join(output_path, "pca_scores.csv"))
        
        loadings_df = pd.DataFrame(loadings, columns=subset_data.columns, index=[f"PC{i}" for i in range(1, len(selected_components)+1)]).T
        loadings_df.to_csv(os.path.join(output_path, "pca_loadings.csv"))
        
        # Save PCA summary
        with open(os.path.join(output_path, "PCA_summary.txt"), "w", encoding="utf-8") as f:
            f.write(f"PCA Summary for {country_name} - {subset_name}\n")
            f.write("="*40 + "\n\n")
            f.write("Eigenvalues:\n")
            for i, val in enumerate(eigenvalues, start=1):
                f.write(f"PC{i}: {val:.4f}\n")
            f.write("\nComponents retained (Kaiser Rule):\n")
            f.write(f"{selected_components}\n")
            f.write(f"Number of components retained: {len(selected_components)}\n\n")
            f.write("Explained Variance Ratio:\n")
            for i, ratio in enumerate(pca.explained_variance_ratio_, start=1):
                f.write(f"PC{i}: {ratio*100:.2f}%\n")
    
    print(f"Processing completed for {country_name}.")

# Process all files
for file_name in os.listdir(input_dir):
    process_country(file_name)

print("All countries processed successfully.")
